In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

In [3]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

# 装载训练集
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [4]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
# 定义网络结构
#  input_size输入特征的大小，hidden_sizeLSTM模块的数量，num_layer表示LSTM的层数
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size = 28,
            hidden_size = 64,
            num_layers = 1,
            batch_first = True
        )
        self.out = torch.nn.Linear(in_features=64,out_features=10)
        self.softmax = torch.nn.Softmax(dim=1)
        
    def forward(self,x):#卷积层只能处理四维的
        #将四维数据改成三维数据（batch,seq_len,feature）
        x = x.view(-1,28,28)
        #output输出三维数据，[batch,seq_len,hidden_size]，包含每个序列的输出结果
        #虽然batch_first=true,但是h_n,c_n的第一个纬度仍然还是num_layers
        #h_n:[num_layer,batch,hidden_size],只包含最后一个序列的输出结果
        #c_n:[num_layer,batch,hidden_size],只包含最后一个序列的输出结果
        output,(h_n,c_n) = self.lstm(x)
        output_in_last_timestep = h_n[-1,:,:]
        x=self.out(output_in_last_timestep)
        x=self.softmax(x)
        return x

In [6]:
LR = 0.0003
# 定义模型
model = LSTM()
# 定义代价函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

In [10]:
def train():
    model.train()#定义模型训练状态，此时dropout起作用
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

def test():
    model.eval()#定义模型测试状态，此时dropout不起作用
    correct = 0
    for i,data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    
    #训练集的准确率
    correct = 0
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item()/len(train_dataset)))

In [11]:
for epoch in range(10):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test acc:0.8124
Train acc:0.8070833333333334
epoch: 1


KeyboardInterrupt: 

In [ ]:
epoch: 0
Test acc:0.8883
epoch: 1
Test acc:0.9008
epoch: 2
Test acc:0.9088
epoch: 3
Test acc:0.9128
epoch: 4
Test acc:0.9138
epoch: 5
Test acc:0.9153
epoch: 6
Test acc:0.916
epoch: 7
Test acc:0.9169
epoch: 8
Test acc:0.9184
epoch: 9
Test acc:0.92